In [1]:
from datetime import date
from glob import glob
import json
import math
import os
import sys
import time

import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import find_nearest_ind

In [4]:
%matplotlib notebook

## Load EEGexp

In [5]:
rec_folder = r"P:\mouse678912\spont_aw_psi_2023-06-22_11-42-00\experiment1\recording1"
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
print(exp.experiment_data)
print(exp.date)

Experiment type: sensory stimulation
['probeB_sorted', 'probeC_sorted', 'probeD_sorted', 'probeF_sorted', 'recording1']
2023-06-22 11:42:00


### Load messages

In [6]:
msg_file = sorted(glob(exp.data_folder + '/**/text.npy', recursive=True))[0]
messages_text = np.load(msg_file).astype(str)
print(messages_text)

['saline injection ' 'saline injection ' 'psi injection ' 'psi injection ']


### Load 'sample_numbers.npy'
A note about the sample numbers: the sampling rate of the saved messages is equal to the highest sampling rate recorded by Open Ephys during the session. For example, if the experiment only records the EEG signals, messages are sampled at 2500 Hz, and you can match the sample numbers of the messages to the sample numbers of the EEG recording. If the experiment also includes Neuropixels, messages are sampled at 30 kHz. In this case, you can match the sample numbers to the AP sample numbers OR convert the sample rate to 2500 Hz and match to the EEG or LFP (smaller file to load).

In [7]:
msgSN_file = sorted(glob(os.path.dirname(msg_file) + '/sample_numbers.npy', recursive=True))[0]
msgSN = np.load(msgSN_file)
print(msgSN)

[78360408 78360408 98185284 98185284]


### Match the sample number with the sync time in seconds

In [11]:
if (len(exp.experiment_data) == 1) and (np.any([True for x in exp.experiment_data if 'recording' in x])):
    print('Using EEG timestamps...')
    message_samples = msgSN
    masterts = np.load(exp.ephys_params['EEG']['timestamps'])
    mastersamples = np.load(os.path.join(os.path.dirname(exp.ephys_params['EEG']['timestamps']), 'sample_numbers.npy'))
elif np.any([True for x in exp.experiment_data if 'probe' in x]):
    print('Using NPX-LFP timestamps...')
    choose_probe = [x for x in exp.experiment_data if 'probe' in x][0].replace('_sorted', '')
    message_samples = (
        msgSN * (exp.ephys_params[choose_probe]['lfp_sample_rate'] / exp.ephys_params[choose_probe]['ap_sample_rate'])
    ).astype(int)
    masterts = np.load(exp.ephys_params[choose_probe]['lfp_timestamps'])
    mastersamples = np.load(
        os.path.join(os.path.dirname(exp.ephys_params[choose_probe]['lfp_timestamps']), 'sample_numbers.npy')
    )

Using NPX-LFP timestamps...


In [12]:
messages_timestamps = masterts[np.array([find_nearest_ind(mastersamples, x) for x in message_samples])]
print(messages_timestamps)

[2522.03747996 2522.03747996 3182.86470898 3182.86470898]
